In [2]:
import numpy as np

In [39]:
from itertools import repeat

l = ['a', 'c', 'e', 'b']
l = [x for item in l for x in repeat(item, 2)]

In [50]:
class Thruster:

    def __init__(self, pos, K):
        self._r = np.array(pos)
        self._K = K
        self._u = np.array([0, 0])

    @property
    def pos_x(self):
        """
        x-position of thruster
        """
        return self._r[0]

    @property
    def pos_y(self):
        """
        y-position of thruster
        """
        return self._r[1]
    
    @property
    def K(self):
        """
        thrust coefficient of thruster
        """
        return self._K

In [51]:
lx = [1.0678, 0.9344, 0.9344, -1.1644, -0.9911, -0.9911] 
ly = [0.0, 0.11, -0.11, 0.0, -0.1644, 0.1644]
K = [0.3763, 0.3901, 0.3776, 0.5641, 0.4799, 0.5588] # Thrust koefficient

In [163]:
from abc import ABC, abstractmethod
DOFS = 3

class AllocationError(Exception):
    """
    AllocationError class
    """

class AllocatorCSAD(ABC):
    """
    Abstract base class for allocation problem
    formulations.
    """

    def __init__(self):
        self._thrusters = []

    @property
    def n_thrusters(self):
        """
        Number of thrusters assigned to this
         allocation problem.
        """
        return len(self._thrusters)
    
    @property
    def n_problem(self):
        """
        Number of unknown variables to be allocated
         in the original (non-relaxed) problem.
        """
        return 2 * self.n_thrusters

    def add_thruster(self, thruster):
        """
        Add a thruster to the allocation problem.
        """
        if isinstance(thruster, Thruster):
            self._thrusters.append(thruster)
        else:
            raise TypeError("Thruster is not of proper type!")
    
    
    def extended_allocation_problem(self):
        """
        Assemble allocation problem into matrix form
        """

        T_e = np.zeros((DOFS, self.n_problem))
        T_e[0, ::2] = 1
        T_e[1, 1::2] = 1
        T_e[2, ::2] = [-thruster.pos_y for thruster in self._thrusters]
        T_e[2, 1::2] = [thruster.pos_x for thruster in self._thrusters]
        
        K_lst = [thruster._K for thruster in self._thrusters]

        K_vec = [x for item in K_lst for x in repeat(item, 2)]

        K_e = np.diag(K_vec)


        return T_e, K_e


    def allocate(self, tau_d):
        """
        Allocate global thrust vector to available thrusters. 
        """
        if self.n_thrusters == 0:
            raise AllocationError(
                """At least one thruster must be added
            to the
             allocator-object before attempting an allocation!"""
            )
        
        T_e, K_e = self.extended_allocation_problem()

        T_e_pseudo = np.transpose(T_e) @ (np.linalg.inv(T_e @ np.transpose(T_e)))

        u_e = np.linalg.inv(K_e) @ T_e_pseudo @ tau_d

        print(u_e)
        u = np.zeros(6)
        alpha = np.zeros(6)

        for i in range(self.n_thrusters):
            u[i] = np.sqrt(u_e[i*2]**2 + u_e[i*2+1]**2)
            alpha[i] = np.arctan(u_e[i*2+1]/u_e[i*2])
        
        return u, alpha


In [164]:
al = AllocatorCSAD()
for i in range(6):
    al.add_thruster(Thruster((lx[i], ly[i]), K[i]))

In [165]:
al.allocate(np.array([10, 5, 2]))

[4.42909026 3.22990888 4.17471299 2.99717075 4.5147717  3.09638853
 2.95455888 0.78361282 3.59163467 1.0462145  2.88065113 0.89849381]
0
1
4.429090264859598
3.229908880664728
5.481710677486631
1
3
4.174712994011355
2.997170745538362
5.139188755073921
2
5
4.514771701190423
3.0963885271041187
5.474558011808066
3
7
2.9545588843585646
0.7836128191178675
3.056708565013678
4
9
3.591634672520406
1.0462145043472013
3.740909569871641
5
11
2.8806511345576067
0.8984938092988938
3.017522507683491


(array([5.48171068, 5.13918876, 5.47455801, 3.05670857, 3.74090957,
        3.01752251]),
 array([0.63008755, 0.62266145, 0.60115587, 0.25925277, 0.2834488 ,
        0.30234408]))

In [4]:
np.clip(4, 1, 3)


3